In [1]:
class OPTNode(AbstractDeclarativeNode):
    def __init__(self):
        self.rho_eq = 1.0
        self.rho_goal = 1.0
        self.rho_lane = 1.0
        self.rho_nonhol = 1.0
        self.rho_w_psi = 1.0
        self.rho_psi = 1.0
        self.maxiter = 500
        self.weight_smoothness = 1.0
        self.weight_smoothness_psi = 1.0

        self.t_fin = 8.0
        self.num = 100
        self.t = self.t_fin/self.num

        tot_time = torch.linspace(0.0, self.t_fin, self.num).to(device)
        tot_time_copy = tot_time.reshape(self.num, 1)
        self.P, self.Pdot, self.Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)

        self.nvar = np.shape(self.P)[1]
        self.A_eq_psi = np.vstack((self.P[0], self.Pdot[0], self.P[-1], self.Pdot[-1]))

        self.cost_smoothness = self.weight_smoothness*np.dot(self.Pddot.T, self.Pddot)
        self.cost_smoothness_v = self.weight_smoothness*np.dot(self.Pddot.T, self.Pddot)
        self.cost_smoothness_psi = self.weight_smoothness_psi*np.dot(self.Pddot.T, self.Pddot)
        self.lincost_smoothness_psi = np.zeros(self.nvar)

        self.P = torch.tensor(self.P, dtype=torch.double).to(device)
        self.Pdot = torch.tensor(self.Pdot, dtype=torch.double).to(device)
        self.Pddot = torch.tensor(self.Pddot, dtype=torch.double).to(device)
        self.A_eq_psi = torch.tensor(self.A_eq_psi, dtype=torch.double).to(device)
        self.cost_smoothness = torch.tensor(self.cost_smoothness, dtype=torch.double).to(device)
        self.cost_smoothness_v = torch.tensor(self.cost_smoothness_v, dtype=torch.double).to(device)
        self.cost_smoothness_psi = torch.tensor(self.cost_smoothness, dtype=torch.double).to(device)
        self.lincost_smoothness_psi = torch.tensor(self.lincost_smoothness_psi, dtype=torch.double).to(device)

        self.rho_mid = 0.01
        self.mid_idx = torch.tensor([ int(self.num/4), int(self.num/2), int(3*self.num/4)]).to(device)
    
    def compute_w_psi(self, x_init, y_init, x_fin, y_fin, x_mid, y_mid, psi, v, lamda_wc, lamda_ws):

        A_w = self.P
        A_w_psi = self.P

        b_wc_psi = torch.tensor(np.cos(psi), dtype=torch.double)
        b_ws_psi = torch.tensor(np.sin(psi), dtype=torch.double)

        temp_x = torch.cumsum(self.P*(v*self.t)[:, np.newaxis] , axis = 0)
        temp_y = torch.cumsum(self.P*(v*self.t)[:, np.newaxis] , axis = 0)

        A_x = temp_x[0:self.num-1]
        A_y = temp_y[0:self.num-1]

        A_x_goal = A_x[-1].reshape(1, self.nvar)
        b_x_goal = torch.tensor([x_fin-x_init], dtype=torch.double)

        A_y_goal = A_y[-1].reshape(1, self.nvar)
        b_y_goal = torch.tensor([y_fin-y_init], dtype=torch.double)


        A_x_mid = A_x[self.mid_idx]
        A_y_mid = A_y[self.mid_idx]

        b_x_mid = torch.tensor(x_mid-x_init, dtype=torch.double)
        b_y_mid = torch.tensor(y_mid-y_init, dtype=torch.double)

        obj_x_goal = self.rho_goal*torch.matmul(A_x_goal.T, A_x_goal)
        linterm_augment_x_goal = -self.rho_goal*torch.matmul(A_x_goal.T, b_x_goal)

        obj_y_goal = self.rho_goal*torch.matmul(A_y_goal.T, A_y_goal)
        linterm_augment_y_goal = -self.rho_goal*torch.matmul(A_y_goal.T, b_y_goal)

        obj_x_mid = self.rho_mid*torch.matmul(A_x_mid.T, A_x_mid)
        linterm_augment_x_mid = -self.rho_mid*torch.matmul(A_x_mid.T, b_x_mid)

        obj_y_mid = self.rho_mid*torch.matmul(A_y_mid.T, A_y_mid)
        linterm_augment_y_mid = -self.rho_mid*torch.matmul(A_y_mid.T, b_y_mid)

        obj_wc_psi = self.rho_w_psi*torch.matmul(A_w_psi.T, A_w_psi)
        linterm_augment_wc_psi = -self.rho_w_psi*torch.matmul(A_w_psi.T, b_wc_psi)

        obj_ws_psi = self.rho_w_psi*torch.matmul(A_w_psi.T, A_w_psi)
        linterm_augment_ws_psi = -self.rho_w_psi*torch.matmul(A_w_psi.T, b_ws_psi)

        cost_wc = obj_wc_psi+obj_x_goal+obj_x_mid
        lincost_wc = -lamda_wc+linterm_augment_x_goal+linterm_augment_wc_psi+linterm_augment_x_mid

        cost_ws = obj_y_goal+obj_ws_psi+obj_y_mid
        lincost_ws = -lamda_ws+linterm_augment_y_goal+linterm_augment_ws_psi+linterm_augment_y_mid

        c_wc_psi = torch.linalg.solve(-cost_wc, lincost_wc)
        c_ws_psi = torch.linalg.solve(-cost_ws, lincost_ws)

        wc = torch.matmul(self.P, c_wc_psi)
        ws = torch.matmul(self.P, c_ws_psi)

        return wc, ws, c_wc_psi, c_ws_psi

    
    def compute_psi(self, wc, ws, b_eq_psi, lamda_psi):

        A_psi = self.P
        b_psi = torch.tensor(np.arctan2(ws, wc), dtype=torch.double)

        obj_psi = self.rho_psi*torch.matmul(A_psi.T, A_psi)
        linterm_augment_psi = -self.rho_psi*torch.matmul(A_psi.T, b_psi)

        cost_psi = self.cost_smoothness_psi+obj_psi+self.rho_eq*torch.matmul(self.A_eq_psi.T, self.A_eq_psi)
        lincost_psi = -lamda_psi+linterm_augment_psi-self.rho_eq*torch.matmul(self.A_eq_psi.T, b_eq_psi)

        sol = torch.linalg.solve(-cost_psi, lincost_psi)

        c_psi = sol[0:self.nvar]

        psi = torch.matmul(self.P, c_psi)
        # self.psidot = np.dot(self.Pdot, c_psi)
        # self.psiddot = np.dot(self.Pddot, c_psi)


        res_psi = torch.matmul(A_psi, c_psi)-b_psi
        res_eq_psi = torch.matmul(self.A_eq_psi, c_psi)-b_eq_psi
        lamda_psi = lamda_psi-self.rho_psi*torch.matmul(A_psi.T, res_psi)-self.rho_eq*torch.matmul(self.A_eq_psi.T, res_eq_psi)


        # self.lamda_psi = self.lamda_psi+0.90*(self.lamda_psi-lamda_psi_old)

        return psi, c_psi, np.linalg.norm(res_psi), np.linalg.norm(res_eq_psi), lamda_psi

    def compute_v(self, v_init, x_init, x_fin, x_mid, y_mid, psi, lamda_v):
        temp_x = torch.cumsum(self.P*(np.cos(psi)*self.t)[:, np.newaxis], axis = 0)
        temp_y = torch.cumsum(self.P*(np.sin(psi)*self.t)[:, np.newaxis], axis = 0)

        A_x = temp_x[0:self.num-1]
        A_y = temp_y[0:self.num-1]

        A_x_goal = A_x[-1].reshape(1, self.nvar)
        b_x_goal = torch.tensor([x_fin-x_init ], dtype=torch.double)

        A_y_goal = A_y[-1].reshape(1, self.nvar)
        b_y_goal = torch.tensor([y_fin-y_init ], dtype=torch.double)

        A_x_mid = A_x[self.mid_idx]
        A_y_mid = A_y[self.mid_idx]

        b_x_mid = torch.tensor(x_mid-x_init, dtype=torch.double)
        b_y_mid = torch.tensor(y_mid-y_init, dtype=torch.double)

        A_vel_init = self.P[0].reshape(1, self.nvar)
        b_vel_init = torch.tensor([v_init], dtype=torch.double)



        obj_x_goal = self.rho_goal*torch.matmul(A_x_goal.T, A_x_goal)
        linterm_augment_x_goal = -self.rho_goal*torch.matmul(A_x_goal.T, b_x_goal)

        obj_y_goal = self.rho_goal*torch.matmul(A_y_goal.T, A_y_goal)
        linterm_augment_y_goal = -self.rho_goal*torch.matmul(A_y_goal.T, b_y_goal)

        obj_x_mid = self.rho_mid*torch.matmul(A_x_mid.T, A_x_mid)
        linterm_augment_x_mid = -self.rho_mid*torch.matmul(A_x_mid.T, b_x_mid)

        obj_y_mid = self.rho_mid*torch.matmul(A_y_mid.T, A_y_mid)
        linterm_augment_y_mid = -self.rho_mid*torch.matmul(A_y_mid.T, b_y_mid)

        obj_v_init = self.rho_eq*torch.matmul(A_vel_init.T, A_vel_init)
        linterm_augment_v_init = -self.rho_eq*torch.matmul(A_vel_init.T, b_vel_init)

        cost = obj_x_goal+obj_y_goal+self.cost_smoothness_v+obj_x_mid+obj_y_mid+obj_v_init
        lincost = -lamda_v+linterm_augment_x_goal+linterm_augment_y_goal+linterm_augment_x_mid+linterm_augment_y_mid+linterm_augment_v_init

        sol = torch.linalg.solve(-cost, lincost)

        # cv = hstack((cv_1, cv_2, sol, cv_10, cv_11))

        v = torch.matmul(self.P, sol)
        # self.vdot = np.dot(self.Pdot, sol)


        res_v_init = torch.matmul(A_vel_init, sol)-b_vel_init
        lamda_v = lamda_v-self.rho_eq*torch.matmul(A_vel_init.T, res_v_init)

        return sol, lamda_v, v

    def optimize(self, x_init, y_init, x_fin, y_fin, v_init, v_fin, psi_init, psidot_init, psi_fin, psidot_fin, x_mid, y_mid):
        v = v_init*torch.ones(self.num)
        psi = psi_init*torch.ones(self.num)


        res_psi = torch.ones(self.maxiter)
        res_w_psi = torch.ones(self.maxiter)
        res_w = torch.ones(self.maxiter)
        res_eq_psi = torch.ones(self.maxiter)
        res_eq = torch.ones(self.maxiter)

        lamda_wc = torch.zeros(self.nvar)
        lamda_ws = torch.zeros(self.nvar)
        lamda_psi = torch.zeros(self.nvar)
        lamda_v = torch.zeros(self.nvar)

        b_eq_psi = torch.hstack((psi_init, psidot_init, psi_fin, psidot_fin))


        for i in range(0, self.maxiter):	


            wc, ws, c_wc_psi, c_ws_psi = self.compute_w_psi(x_init, y_init, x_fin, y_fin, x_mid, y_mid, psi, v, lamda_wc, lamda_ws)
            psi, c_psi, res_psi[i], res_eq_psi[i], lamda_psi = self.compute_psi(wc, ws, b_eq_psi, lamda_psi)
            c_v, lamda_v, v = self.compute_v(v_init, x_init, x_fin, x_mid, y_mid, psi, lamda_v)



            res_wc = wc-torch.cos(psi)
            res_ws = ws-torch.sin(psi)
            self.A_w = self.P

            # lamda_wc_old = self.lamda_wc
            # lamda_ws_old = self.lamda_ws


            lamda_wc = lamda_wc-self.rho_w_psi*torch.matmul(self.A_w.T, res_wc)
            lamda_ws = lamda_ws-self.rho_w_psi*torch.matmul(self.A_w.T, res_ws)

            # self.lamda_wc = self.lamda_wc+0.90*(self.lamda_wc-lamda_wc_old)
            # self.lamda_ws = self.lamda_ws+0.90*(self.lamda_ws-lamda_ws_old)


            res_w[i] = torch.linalg.norm( torch.hstack(( res_wc, res_ws ))  )
            # res_eq[i] = np.linalg.norm( np.hstack((  res_eq_x_vec, res_eq_y_vec     ))  )
            # res_obs_lane[i] = np.linalg.norm( np.hstack(( res_x_obs_vec_lane, res_y_obs_vec_lane         ))  )


        primal_sol = torch.hstack(( c_psi, c_v         ))
        dual_sol = torch.hstack((  lamda_wc, lamda_ws, lamda_psi, lamda_v       ))

        return primal_sol, dual_sol, res_eq_psi, res_w, c_v
    
    def objective(self, x_init, y_init, x_fin, y_fin, v_init, v_fin, psi_init, psidot_init, psi_fin, psidot_fin, x_mid, y_mid):
        primal_sol, dual_sol, res_eq_psi, res_w, c_v = self.optimize(x_init, y_init, x_fin, y_fin, v_init, v_fin, psi_init, psidot_init, psi_fin, psidot_fin, x_mid, y_mid)
        
        c_psi = primal_sol[0:nvar]
        c_v = primal_sol[nvar:2*nvar]
        v_new = torch.matmul(self.P, c_v)
        psi_new = torch.matmul(self.P, c_psi)
        psidot_new = torch.matmul(self.Pdot, c_psi)
        x_temp = x_init + torch.cumsum(v*torch.cos(psi_new)*self.t, dim=0)
        y_temp = y_init + torch.cumsum(v*torch.sin(psi_new)*self.t, dim=0)
        x = torch.hstack((x_init, x_temp[0:-1]))
        y = torch.hstack((y_init, y_temp[0:-1]))
        
        cost_final_pos = 0.5*self.rho_goal*((x[-1]-x_fin)**2+(y[-1]-y_fin)**2)
        cost_psi_term = 0.5*self.rho_eq*( ( psi_new[0]-psi_init)**2+( psidot_new[0]-psidot_init)**2+( psi_new[-1]-psi_fin)**2+( psidot_new[-1]-psidot_fin)**2 )
        cost_v_term = 0.5*self.rho_eq*(v[0]-v_init)**2
        cost_mid_term = 0.5*self.rho_mid*( torch.sum(( x[self.mid_idx]-x_mid  )**2)+torch.sum(( y[self.mid_idx]-y_mid  )**2) )

        cost = cost_final_pos+cost_psi_term+cost_v_term+cost_mid_term

        return cost 

    def solve(self, b):
        batch_size, _ = b.size()
        y = torch.zeros(batch_size, 2*self.num)
        mid_size = self.mid_idx.size()[0]
        for i in range(batch_size):
            x_init, y_init, x_fin, y_fin, v_init, v_fin = b[i][0], b[i][1], b[i][2], b[i][3], b[i][4], b[i][5]
            psi_init, psidot_init, psi_fin, psidot_fin = b[i][6], b[i][7], b[i][8], b[i][9]
            x_mid = b[i][10:10+mid_size]
            y_mid = b[i][10+mid_size:]
            primal_sol, dual_sol, res_eq_psi, res_w, c_v = self.optimize(x_init, y_init, x_fin, y_fin, v_init, v_fin, psi_init, psidot_init, psi_fin, psidot_fin, x_mid, y_mid)
            c_psi = primal_sol[0:nvar]
            c_v = primal_sol[nvar:2*nvar]
            v_new = torch.matmul(self.P, c_v)
            psi_new = torch.matmul(self.P, c_psi)
            x_temp = x_init + torch.cumsum(v*torch.cos(psi_new)*self.t, dim=0)
            y_temp = y_init + torch.cumsum(v*torch.sin(psi_new)*self.t, dim=0)
            x_ret = torch.hstack((x_init, x_temp[0:-1]))
            y_ret = torch.hstack((y_init, y_temp[0:-1]))
            y[i, :] = torch.hstack((x_ret, y_ret))
        return y, None

NameError: name 'AbstractDeclarativeNode' is not defined